### A* 

this algorithm is an extension of the bfs.

1. cost is added to the tuple of the PriorityQueue
```
q = PriorityQueue()
q.put((100,20)# implemented as (key,value)
q.put((1,23))
q.get() # returns (1,23)
```

2. cost is modeled as the sum of the action + the cost as heuristic ( distance of current position to the goal) 

In [12]:
import numpy as np
from queue import PriorityQueue
from enum import Enum

In [32]:
#################
q = PriorityQueue()
q.put((100,20))# implemented as (key,value)
q.put((1,23))
q.get() # returns (1,23)

################

(1, 23)

In [13]:
position = (1,0)
goal_position = (3,3)

def heuristic(position, goal_postition):
    h = np.linalg.norm([goal_position[0] - position[0],
                        goal_position[1] - position[0]])
    return h

In [14]:
heuristic(position, goal_position)

2.8284271247461903

In [15]:
class Action(Enum):
    """
    An action is represented by a 3 element tuple.
    
    The first 2 values are the delta of the action relative
    to the current grid position. The third and final value
    is the cost of performing the action.
    """
    LEFT = (0, -1, 1)
    RIGHT = (0, 1, 1)
    UP = (-1, 0, 1)
    DOWN = (1, 0, 1)
    
    def __str__(self):
        if self == self.LEFT:
            return '<'
        elif self == self.RIGHT:
            return '>'
        elif self == self.UP:
            return '^'
        elif self == self.DOWN:
            return 'v'
    
    @property
    def cost(self):
        return self.value[2]
    
    @property
    def delta(self):
        return (self.value[0], self.value[1])
            
    
def valid_actions(grid, current_node):
    """
    Returns a list of valid actions given a grid and current node.
    """
    valid = [Action.UP, Action.LEFT, Action.RIGHT, Action.DOWN]
    n, m = grid.shape[0] - 1, grid.shape[1] - 1
    x, y = current_node
    
    # check if the node is off the grid or
    # it's an obstacle
    
    if x - 1 < 0 or grid[x-1, y] == 1:
        valid.remove(Action.UP)
    if x + 1 > n or grid[x+1, y] == 1:
        valid.remove(Action.DOWN)
    if y - 1 < 0 or grid[x, y-1] == 1:
        valid.remove(Action.LEFT)
    if y + 1 > m or grid[x, y+1] == 1:
        valid.remove(Action.RIGHT)
        
    return valid

def visualize_path(grid, path, start):
    sgrid = np.zeros(np.shape(grid), dtype=np.str)
    sgrid[:] = ' '
    sgrid[grid[:] == 1] = 'O'
    
    pos = start
    
    for a in path:
        da = a.value
        sgrid[pos[0], pos[1]] = str(a)
        pos = (pos[0] + da[0], pos[1] + da[1])
    sgrid[pos[0], pos[1]] = 'G'
    sgrid[start[0], start[1]] = 'S'  
    return sgrid

In [55]:

queue_status_dic = {}
 
def a_star(grid, h, start, goal):
    cnt = 0
    path = []
    path_cost = 0
    queue = PriorityQueue()
    queue.put((0, start))
    visited = set(start)

    branch = {}
    found = False
    
    
    
    
    while not queue.empty():
        cnt +=1 
        item = queue.get()
        current_node = item[1]
        if current_node == start:
            current_cost = 0.0
        else:              
            current_cost = branch[current_node][0]
            
        if current_node == goal:        
            print('Found a path.')
            found = True
            break
        else:
            for action in valid_actions(grid, current_node):
                # get the tuple representation
                da = action.delta
                next_node = (current_node[0] + da[0], current_node[1] + da[1])
                # DONE: calculate branch cost (action.cost + g)
                h = heuristic(next_node, goal)
                # DONE: calculate queue cost (action.cost + g + h)
                g = action.cost
                
                """
                this approach is wrong for 2 reasons:
                
                1.  !! ONLY ACTIONS COST ARE EFFECTIVE COSTS ( THESE ARE EVALUATED IN THE BRANCH ) !!  
                2.  the cost (c_new = c + h + g ) is the one determining the priority in the que but it is not an effective cost.
                
                """
                
                #if current_node == start:
                    #branch_cost = h + g
                #else:
                   #branch_cost = branch[current_node][0] + h +g 
                branch_cost = current_cost + g
                queue_cost = current_cost + h + g 
                
                if next_node not in visited:                
                    visited.add(next_node)               
                    branch[next_node] = (branch_cost, current_node, action)
                    queue.put((queue_cost, next_node))    
                    
                queue_status_dic[cnt] = list(queue.queue)
    if found:
        # retrace steps
        n = goal
        path_cost = branch[n][0]
        while branch[n][1] != start:
            path.append(branch[n][2])
            n = branch[n][1]
        path.append(branch[n][2])
    else:
        print('**********************')
        print('Failed to find a path!')
        print('**********************')
        
    return path[::-1], path_cost

In [56]:
start = (0, 0)
goal = (4, 4)

grid = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0],
])

In [57]:
path, cost = a_star(grid, heuristic, start, goal)
print(path, cost)

Found a path.
([<Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.LEFT: (0, -1, 1)>], 10.0)


In [58]:
visualize_path(grid, path, start)

array([['S', 'O', ' ', ' ', ' ', ' '],
       ['>', '>', '>', 'v', ' ', ' '],
       [' ', 'O', ' ', '>', '>', 'v'],
       [' ', ' ', ' ', 'O', 'O', 'v'],
       [' ', ' ', ' ', 'O', 'G', '<']], dtype='|S1')

In [62]:


for qsd in queue_status_dic.items():
    print("------------")
    print(qsd)

------------
(1, [(3.8284271247461903, (1, 0))])
------------
(2, [(3.414213562373095, (2, 0)), (6.242640687119285, (0, 0)), (4.82842712474619, (1, 1))])
------------
(3, [(3.0, (3, 0)), (6.242640687119285, (0, 0)), (4.82842712474619, (1, 1))])
------------
(4, [(4.0, (3, 1)), (5.414213562373095, (4, 0)), (4.82842712474619, (1, 1)), (6.242640687119285, (0, 0))])
------------
(5, [(4.82842712474619, (1, 1)), (5.0, (3, 2)), (6.242640687119285, (0, 0)), (5.414213562373095, (4, 0)), (6.414213562373095, (4, 1))])
------------
(6, [(5.0, (3, 2)), (5.414213562373095, (4, 0)), (6.242640687119285, (0, 0)), (6.414213562373095, (4, 1)), (5.82842712474619, (1, 2))])
------------
(7, [(5.414213562373095, (4, 0)), (5.82842712474619, (1, 2)), (6.242640687119285, (0, 0)), (6.414213562373095, (4, 1)), (7.414213562373095, (2, 2)), (7.414213562373095, (4, 2))])
------------
(8, [(5.82842712474619, (1, 2)), (6.414213562373095, (4, 1)), (6.242640687119285, (0, 0)), (7.414213562373095, (4, 2)), (7.414213562